<a href="https://colab.research.google.com/github/nikeshkumar7/colab/blob/main/plant_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image,ImageTk
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(256, 256))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
def predict_disease(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_label = class_indices_reverse[class_index]
    confidence = prediction[0, class_index]

    print(f"Prediction: {class_label}, Confidence: {confidence}")
    return class_label

In [ ]:
dataset_path = "C:/Users/DESKTOP/Desktop/plant/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"

In [ ]:
train_datagen = ImageDataGenerator(
    zoom_range=0.5,
    shear_range=0.3,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(dataset_path, "train"),
    target_size=(256, 256),
    batch_size=32
)

Found 57422 images belonging to 32 classes.


In [ ]:
#from PIL import Image, ImageTk
#from tkinter import filedialog

class PlantDiseaseClassifierGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Plant Disease Classifier")

        # configure the Frame
        self.frame = tk.Frame(self.root, padx=200, pady=200)
        self.frame.pack()

        # Buttons
        self.browse_image_button = tk.Button(self.frame, text="Browse Image", command=self.browse_image)
        self.browse_image_button.pack(side=tk.LEFT)

        self.browse_directory_button = tk.Button(self.frame, text="Browse Directory", command=self.browse_directory)
        self.browse_directory_button.pack(side=tk.LEFT)

        self.predict_button = tk.Button(self.frame, text="Predict", command=self.predict_image)
        self.predict_button.pack(side=tk.RIGHT)

        self.clear_button = tk.Button(self.frame, text="Clear", command=self.clear_result)
        self.clear_button.pack(side=tk.RIGHT)

        # Label to display the prediction result
        self.result_label = tk.Label(self.root, text="")
        self.result_label.pack()

        # Initialize the image label
        self.image_label = tk.Label(self.root)

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", ".png;*.jpg;*.jpeg")])
        self.image_path = file_path

        # Display the selected image on the GUI
        img = Image.open(self.image_path)
        img = img.resize((300, 300), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        self.image_label.config(image=img)
        self.image_label.image = img
        self.image_label.pack()

    def browse_directory(self):
        directory_path = filedialog.askdirectory()
        self.image_path = directory_path
        self.result_label.config(text="Selected directory: " + directory_path)

    def predict_image(self):
        if hasattr(self, 'image_path'):
            if os.path.isfile(self.image_path):
                prediction = predict_disease(self.image_path)
                self.result_label.config(text=f"The image belongs to {prediction}")
            elif os.path.isdir(self.image_path):
                self.result_label.config(text="Predicting diseases in the directory...")
                self.predict_directory()
            else:
                self.result_label.config(text="Please select a valid image or directory.")
        else:
            self.result_label.config(text="Please select an image or directory first.")

    def predict_directory(self):
        # Get the list of images in the directory
        images = [f for f in os.listdir(self.image_path) if os.path.isfile(os.path.join(self.image_path, f))]

        for image_name in images:
            image_path = os.path.join(self.image_path, image_name)
            prediction = predict_disease(image_path)
            print(f"The image {image_name} belongs to {prediction}")

        self.result_label.config(text="Prediction complete.")

    def clear_result(self):
        self.image_label.pack_forget()
        self.result_label.config(text="")
        self.image_path = None

# Tkinter GUI
import tkinter as tk
import os

root = tk.Tk()
app = PlantDiseaseClassifierGUI(root)
root.mainloop()
